In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class SOM:
    def __init__(self, width, height, dim):
        self.num_iters = 100
        self.width = width
        self.height = height
        self.dim = dim
        self.node_locs = self.get_locs()
        
        nodes = tf.Variable(tf.random_normal([width*height, dim]))
        self.nodes = nodes
        
        x = tf.placeholder(tf.float32, [dim])
        iter = tf.placeholder(tf.float32)
        
        self.x = x
        self.iter = iter
        
        bmu_loc = self.get_bmu_loc(x)
        
        self.propagate_nodes = self.get_propagation(bmu_loc, x, iter)

In [5]:
    def get_propagation(self, bmu_loc, x, iter):
        num_nodes = self.width * self.height
        rate = 1.0 - tf.div(iter, self.num_iters)
        alpha = rate * 0.5
        sigma = rate * tf.to_float(tf.maximum(self.width, self.height)) / 2.
        expanded_bmu_loc = tf.expand_dims(tf.to_float(bmu_loc), 0)
        sqr_dists_from_bmu = tf.reduce_sum(
            tf.square(tf.subtract(expanded_bmu_loc, self.node_locs)), 1)
        neigh_factor = tf.stack([tf.tile(tf.slice(rate, [i], [1]), [self.dim]) for i in range(num_nodes)])
        nodes_diff = tf.multiply(
            rate_factor,
            tf.subtract(tf.stack([x for i in range(num_nodes)]), self.nodes))
        update_nodes = tf.add(self.nodes, nodes_diff)
        return tf.assign(self.nodes, update_nodes)

In [8]:
    def get_bmu_loc(self, x):
        expanded_x = tf.expand_dims(x, 0)
        sqr_diff = tf.square(tf.subtract(expanded_x, self.nodes))
        dists = tf.reduce_sum(sqr_diff, 1)
        bmu_idx = tf.argmin(dists, 0)
        bmu_loc = tf.stack([tf.mod(bmu_idx, self.width), tf.div(bmu_idx, self.width)])
        return bmu_loc

In [10]:
    def get_locs(self):
        locs = [[x, y]
               for y in range(self.height)
               for x in range(self.width)]
        return tf.to_float(locs)

In [11]:
    def train(self, data):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initalizer())
            for i in range(self.num_iters):
                for data_x in data:
                    sess.run(self.propagate_nodes, feed_dict={self.x: data_x, self.iter: i})
            centroid_grid = [[] for i in range(self.width)]
            self.nodes_val = list(sess.run(self.nodes))
            self.locs_val = list(sess.run(self.nodes_locs))
            for i, l in enumerate(self.locs_val):
                centroid_grid[int(1[0])].append(self.nodes_val[i])
            self.centroid_grid = centroid_grid

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import sompy

colors = np.array(
     [[0., 0., 1.],
      [0., 0., 0.95],
      [0., 0.05, 1.],
      [0., 1., 0.],
      [0., 0.95, 0.],
      [0., 1, 0.05],
      [1., 0., 0.],
      [1., 0.05, 0.],
      [1., 0., 0.05],
      [1., 1., 0.]])

#som = sompy.SOMFactory.build(4, 4, 3)
mapsize = [100, 100]
som = sompy.SOMFactory.build(colors, mapsize, mask=None, mapshape='planar', lattice='rect', 
                             normalization='var', initialization='pca', neighborhood='gaussian', 
                             training='batch', name='sompy')
#som.train(colors)
#som.train(n_job=1, verbose='info')
som.train(n_job=1, verbose='info')
v = sompy.mapview.View2DPacked(100, 100, 'test',text_size=8)
v.show(som, what='codebook', which_dim="all", cmap=None, col_sz=6) #which_dim='all' default
# v.save('2d_packed_test')

 Training...
 pca_linear_initialization took: 0.090000 seconds
 Rough training...
 radius_ini: 13.000000 , radius_final: 3.250000, trainlen: 30000

 epoch: 1 ---> elapsed time:  2.465000, quantization error: 0.259720

 epoch: 2 ---> elapsed time:  2.504000, quantization error: 0.047691

 epoch: 3 ---> elapsed time:  2.185000, quantization error: 0.029811

 epoch: 4 ---> elapsed time:  2.144000, quantization error: 0.015312

 epoch: 5 ---> elapsed time:  2.150000, quantization error: 0.010758

 epoch: 6 ---> elapsed time:  2.139000, quantization error: 0.009396

 epoch: 7 ---> elapsed time:  2.167000, quantization error: 0.008428

 epoch: 8 ---> elapsed time:  2.152000, quantization error: 0.010138

 epoch: 9 ---> elapsed time:  2.231000, quantization error: 0.008703

 epoch: 10 ---> elapsed time:  2.238000, quantization error: 0.008652

 epoch: 11 ---> elapsed time:  2.207000, quantization error: 0.008303

 epoch: 12 ---> elapsed time:  2.323000, quantization error: 0.008311

 epoch: 1